Data augmentation should not be included in this model because X-ray scans are only taken in a specific orientation, and variations such as flips and rotations will not exist in real X-ray images.

In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Concatenate #Used to concatenat the feature outputs from two models together
from keras.layers import Input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.applications import MobileNetV2, DenseNet169
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
input_shape=(224,224,3)
input_layer=Input(shape=input_shape)

mobilenet_base = MobileNetV2(weights='imagenet', input_shape=input_shape, include_top=False)
densenet_base = DenseNet169(weights='imagenet', input_shape=input_shape, include_top=False)

51877672/51877672 [==============================] - 0s 0us/step


In [3]:
mobilenet_base.summary()
# /content/drive/MyDrive/collab_mount/chest_xray

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [4]:
densenet_base.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_3[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

In [5]:
CLASSES=2

for layer in mobilenet_base.layers:
  layer.trainable=False

for layer in densenet_base.layers:
  layer.trainable=False

model_mobilenet = mobilenet_base(input_layer)
model_mobilenet2 = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet2)

model_densenet = densenet_base(input_layer)
model_densenet2 = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet2)

In [6]:
merged=Concatenate()([output_mobilenet, output_densenet])

In [7]:
x = BatchNormalization()(merged)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

stacked_model = Model(inputs = input_layer, outputs = x)

In [8]:
stacked_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [9]:
stacked_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)           2257984   ['input_1[0][0]']             
 tional)                                                                                          
                                                                                                  
 densenet169 (Functional)    (None, 7, 7, 1664)           1264288   ['input_1[0][0]']             
                                                          0                                       
                                                                                              

In [ ]:
#os.listdir('/content/drive/MyDrive/collab_mount/chest_xray')

In [ ]:
#!rm -rf /content/drive/MyDrive/collab_mount/chest_xray/.DS_Store

In [10]:
WIDTH = 224
HEIGHT = 224
BATCH_SIZE = 16
TRAIN_DIR = r'/content/drive/MyDrive/collab_mount/chest_xray/train'
VAL_DIR = r'/content/drive/MyDrive/collab_mount/chest_xray/val'
TEST_DIR = r'/content/drive/MyDrive/collab_mount/chest_xray/test'

# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(HEIGHT, WIDTH),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary')

val_generator = validation_datagen.flow_from_directory(VAL_DIR,
                                                       target_size=(HEIGHT, WIDTH),
                                                       batch_size=BATCH_SIZE,
                                                       class_mode='binary')

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size=(HEIGHT, WIDTH),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='binary')

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
VALIDATION_STEPS = val_generator.n//val_generator.batch_size
STEPS_PER_EPOCH, VALIDATION_STEPS

In [ ]:
stacked_model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_generator,
    validation_steps=VALIDATION_STEPS, verbose=1
)

In [ ]:
img = image.load_img(r'/path/to/image.jpeg')
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = stacked_model.predict(x)[0]
print(pred)
# print out actual image class